<a href="https://colab.research.google.com/github/hooroobaby/DSonCyberSecurity/blob/main/111356043_H_StaticAnalysis_242variants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%bash
# MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
# MINICONDA_PREFIX=/usr/local
# wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
# chmod +x $MINICONDA_INSTALLER_SCRIPT
# ./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

# conda install --channel defaults conda python=3.8 --yes
# conda update --channel defaults --all --yes

# import sys
# sys.path.append("/usr/local/lib/python3.8/site-packages")


### Static Analysis

Since we cannot share real malware binary with you. We give you the pefile analysis results (by using pefile dump_dict() function) and save the result in a json/pickle file. You have to practice writing a parser before analyzing the given pefile dump file.

dump file #1 worm_unpacked_dll_call.json
https://drive.google.com/uc?id=17TkMc-xXNyQ4muO9hmNGPrGb_mNfqlS9

dump file #2 worm_unpacked_pe_dump.pickle
https://drive.google.com/uc?id=1wLdkVEUsblApCaV2IJbu7XSP6agVUz7-

The first one has less and clean info, but the second one might contain more info for latter analysis.

Below is the "half" information of this dataset. Or you can view it in google spreadsheets.

https://docs.google.com/spreadsheets/d/106IAfoDXZX2FZ5ImiGEQ9iqD-uPaVcJzKscmMTEgr-s/edit?usp=sharing

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
!wget https://drive.google.com/uc?id=1N32lXC-qhr6Z4jwBaROtyowLisddYLys -O worm.csv

--2023-04-15 08:17:54--  https://drive.google.com/uc?id=1N32lXC-qhr6Z4jwBaROtyowLisddYLys
Resolving drive.google.com (drive.google.com)... 108.177.13.102, 108.177.13.138, 108.177.13.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.13.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/m189u6i0fb4a2mf76iqtdjcfped7cq9r/1681546650000/17174434063503717705/*/1N32lXC-qhr6Z4jwBaROtyowLisddYLys?uuid=1068e176-eb62-405c-b13b-a7a5ed0f4554 [following]
--2023-04-15 08:17:54--  https://doc-0k-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/m189u6i0fb4a2mf76iqtdjcfped7cq9r/1681546650000/17174434063503717705/*/1N32lXC-qhr6Z4jwBaROtyowLisddYLys?uuid=1068e176-eb62-405c-b13b-a7a5ed0f4554
Resolving doc-0k-38-docs.googleusercontent.com (doc-0k-38-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to doc-0

In [ ]:
_df = pd.read_csv("worm.csv")
_df

,MD5,Microsoft,noFamilyName,UPXpacked,skip
0,ffdf141aded498e641b45b0e2188e360,Backdoor:Win32/Berbew.DR,0.0,0,0
1,fff738fcbe3244b4048edb3e289dd450,Backdoor:Win32/Berbew.DR,0.0,0,0
2,ffe268ffc185abe5b68970e45eb74810,Backdoor:Win32/Berbew.I,0.0,0,0
3,fff6a4b8265ea0a0d778697c4096c230,Backdoor:Win32/Delf,0.0,0,1
4,fff1517aebae3c980f13efd7edea2480,Backdoor:Win32/Delf.DU,0.0,0,0
...,...,...,...,...,...
237,fffed384aeba28e1474467058b4756a0,NaN,NaN,0,0
238,ffff2a37e92c2ae42c5b8230c1c0c8e0,NaN,NaN,1,0
239,ffff31ad7e82b8cfe50f5ba21e13a450,NaN,NaN,0,0
240,ffff90de328dc28095d7610c18b20130,NaN,NaN,0,0


In [ ]:
_df.Microsoft.isna().value_counts()

True     123
False    119
Name: Microsoft, dtype: int64

In [ ]:
_df[_df.noFamilyName == 1].value_counts()

MD5                               Microsoft                      noFamilyName  UPXpacked  skip
ffdb2677884ac3b6feb1b436747b5f70  Worm:AutoIt/Autorun.K          1.0           1          0       1
ffdcf8a72484358fdba9882dd3c195c0  VirTool:INF/Autorun!rfn        1.0           1          0       1
ffdde3583746c242307cf4c2741f5ce0  Exploit:HTML/IframeRef.gen     1.0           0          1       1
ffe63d41ff66a0dfe31762471171f950  Backdoor:Win32/IRCbot          1.0           0          0       1
ffeaedd146b09f7434bffcf58d60ad20  Exploit:Win32/ShellCode.gen!B  1.0           0          0       1
ffeb34f7748ac1d42a9ea9e522040480  VirTool:INF/Autorun!rfn        1.0           1          0       1
fff0203b682aa62f1d93c7fbc37915f0  Backdoor:Win32/PcClient.ZR     1.0           1          0       1
fff06815cd59af9e269964c12c98d5d0  Exploit:HTML/IframeRef.gen     1.0           0          1       1
fff14238f5f8c79c7de26e25be5ed380  Trojan:HTML/Redirector         1.0           0          1       1
fff43

You can see that there are 242 malware, and almost half of them (116) are labeled by Microsoft. But 10 of them are labeled badly, i.e., no malware family are included in the labels. Some of them are UPXpacked (but do not worry, I have unpacked them for you). Some them are borken files so that they cannot be analyzed (I marked them as 'skip').

Now, lets review some pefile analysis, and at last, see if you can do some classification or clustering on this malware dataset.

### Download dump data #1


In [ ]:
# Here is the 1st dump json file.
!wget https://drive.google.com/uc?id=17TkMc-xXNyQ4muO9hmNGPrGb_mNfqlS9 -O worm_unpacked_dll_call.json

--2023-04-15 08:17:54--  https://drive.google.com/uc?id=17TkMc-xXNyQ4muO9hmNGPrGb_mNfqlS9
Resolving drive.google.com (drive.google.com)... 108.177.13.102, 108.177.13.138, 108.177.13.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.13.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/91ccuf4a22qqdi9emcc9ksdbcoaubk7b/1681546650000/17174434063503717705/*/17TkMc-xXNyQ4muO9hmNGPrGb_mNfqlS9?uuid=196a8d35-8003-4e6d-b1ef-378fccebe51c [following]
--2023-04-15 08:17:54--  https://doc-0c-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/91ccuf4a22qqdi9emcc9ksdbcoaubk7b/1681546650000/17174434063503717705/*/17TkMc-xXNyQ4muO9hmNGPrGb_mNfqlS9?uuid=196a8d35-8003-4e6d-b1ef-378fccebe51c
Resolving doc-0c-38-docs.googleusercontent.com (doc-0c-38-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to doc-0

In [ ]:
# The size should be 363255 bytes.
!ls -al

total 4604
drwxr-xr-x 1 root root    4096 Apr 15 04:34 .
drwxr-xr-x 1 root root    4096 Apr 15 04:33 ..
drwxr-xr-x 4 root root    4096 Apr 13 13:29 .config
-rw-r--r-- 1 root root 1477416 Oct 28 17:30 putty32.exe
drwxr-xr-x 1 root root    4096 Apr 13 13:30 sample_data
-rw-r--r-- 1 root root   12481 Apr 15 08:17 worm.csv
-rw-r--r-- 1 root root  363255 Apr 15 08:17 worm_unpacked_dll_call.json
-rw-r--r-- 1 root root 2838048 Apr 15 08:17 worm_unpacked_pe_dump.pickle


In [ ]:
# pefile information is stored in a json file.
# You can retrive it by using the following code.
# I convert the file to a python dictionary

import json

with open('worm_unpacked_dll_call.json') as json_file:
    worm_dict = json.load(json_file)

    # Print the type of data variable
    print("Type:", type(worm_dict))

Type: <class 'dict'>


In [ ]:
# the dictionary key is the index of the malware
# they are the MD5 value of the malware samples
worm_dict.keys()

dict_keys(['ffff5099125bceaea61251684fbdb770', 'fff80ae9338aec1e979ad1ddeb060560', 'fff000bb386ed5fca21866fab5172500', 'fff71cb8da44b471b448dad8d3c1ac20', 'ffe894389225f1d102769d89261acda0', 'fffda772e9682b5cd713a1aff6bebe71', 'fff1a3306e69df859b43e71cc2671f00', 'fffd904f5de5d9ef529b07b3bcf6f5b0', 'ffde11ff4926d17df917040683095b50', 'fff27dfe35219a2f7ab157fec9a6da40', 'ffe91510f165d479db6492d973ade310', 'ffdb3c00f7cc7c25f8240eed3a1d4a50', 'ffe189289e765a331b9b4f04fbfc7bb0', 'ffe4dabd0d2e8075aee4e7db15ce0d80', 'ffe5eaa82debc744b9c9c900fa22bc70', 'ffede07462c60920e4d59e84b65b7030', 'ffe268ffc185abe5b68970e45eb74810', 'fff2e49743064367fc7612168d185480', 'ffe19ece6b385773c362198940f76460', 'fff3236a1ded79b7c1138fca11971f60', 'fff1cc37ea691d75e793c642f6da3340', 'fff9d17e467f93121d15481266a63950', 'ffddcf15d798e8e22b6ec59232ab1fc0', 'ffeb34f7748ac1d42a9ea9e522040480', 'ffddeedc483f0ab09ddc5cb55d93f8f0', 'ffeeb351089c255e152dd2c79c55b250', 'fff26188530a82f3667c66da19168f20', 'ffff0fb5fed019a0

In [ ]:
# file #1 shows the dynamic library and function calls used by the malware.
worm_dict['ffff5099125bceaea61251684fbdb770']

{'kernel32.dll': ['CloseHandle',
  'CreateFileA',
  'CreateMutexA',
  'ExitProcess',
  'FreeLibrary',
  'GetCommandLineA',
  'GetCurrentThreadId',
  'GetDiskFreeSpaceA',
  'GetFileSize',
  'GetFileType',
  'GetLastError',
  'GetModuleFileNameA',
  'GetModuleHandleA',
  'GetProcAddress',
  'GetProcessHeap',
  'GetStdHandle',
  'GetSystemTime',
  'GetVersionExA',
  'HeapAlloc',
  'HeapFree',
  'HeapReAlloc',
  'LoadLibraryA',
  'LocalAlloc',
  'RaiseException',
  'ReadFile',
  'RtlUnwind',
  'SetEndOfFile',
  'SetFilePointer',
  'TlsGetValue',
  'TlsSetValue',
  'UnhandledExceptionFilter',
  'WaitForSingleObject',
  'WriteFile'],
 'user32.dll': ['CharNextA',
  'GetWindowLongA',
  'MessageBoxA',
  'SetWindowLongA',
  'ShowWindow'],
 'advapi32.dll': ['RegCloseKey',
  'RegCreateKeyExA',
  'RegFlushKey',
  'RegOpenKeyExA',
  'RegQueryValueExA',
  'RegSetValueExA']}

### Download dump data #2


In [ ]:
# Here is the 2nd dump pickle file.
!wget https://drive.google.com/uc?id=1wLdkVEUsblApCaV2IJbu7XSP6agVUz7- -O worm_unpacked_pe_dump.pickle

--2023-04-15 08:17:55--  https://drive.google.com/uc?id=1wLdkVEUsblApCaV2IJbu7XSP6agVUz7-
Resolving drive.google.com (drive.google.com)... 108.177.13.102, 108.177.13.138, 108.177.13.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.13.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/21m9b58r3roa4ti2f9iig88u642f4770/1681546650000/17174434063503717705/*/1wLdkVEUsblApCaV2IJbu7XSP6agVUz7-?uuid=c41f3295-3d04-484c-8c0b-957d05b75bef [following]
--2023-04-15 08:17:55--  https://doc-0c-38-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/21m9b58r3roa4ti2f9iig88u642f4770/1681546650000/17174434063503717705/*/1wLdkVEUsblApCaV2IJbu7XSP6agVUz7-?uuid=c41f3295-3d04-484c-8c0b-957d05b75bef
Resolving doc-0c-38-docs.googleusercontent.com (doc-0c-38-docs.googleusercontent.com)... 173.194.217.132, 2607:f8b0:400c:c13::84
Connecting to doc-0

In [ ]:
!ls -al

total 4604
drwxr-xr-x 1 root root    4096 Apr 15 04:34 .
drwxr-xr-x 1 root root    4096 Apr 15 04:33 ..
drwxr-xr-x 4 root root    4096 Apr 13 13:29 .config
-rw-r--r-- 1 root root 1477416 Oct 28 17:30 putty32.exe
drwxr-xr-x 1 root root    4096 Apr 13 13:30 sample_data
-rw-r--r-- 1 root root   12481 Apr 15 08:17 worm.csv
-rw-r--r-- 1 root root  363255 Apr 15 08:17 worm_unpacked_dll_call.json
-rw-r--r-- 1 root root 2838048 Apr 15 08:17 worm_unpacked_pe_dump.pickle


### 由於python version>3.8，所以pickle.HIGHEST_PROTOCOL為5

In [ ]:
import pickle

In [ ]:
!python --version
print(pickle.HIGHEST_PROTOCOL) # should be 4

Python 3.9.16
5


In [ ]:
with open('worm_unpacked_pe_dump.pickle', 'rb') as f:
    m_dict = pickle.load(f)
print(m_dict.keys())

dict_keys(['ffff5099125bceaea61251684fbdb770', 'fff80ae9338aec1e979ad1ddeb060560', 'fff000bb386ed5fca21866fab5172500', 'fff71cb8da44b471b448dad8d3c1ac20', 'ffe894389225f1d102769d89261acda0', 'fffda772e9682b5cd713a1aff6bebe71', 'fff1a3306e69df859b43e71cc2671f00', 'fffd904f5de5d9ef529b07b3bcf6f5b0', 'ffde11ff4926d17df917040683095b50', 'fff27dfe35219a2f7ab157fec9a6da40', 'ffe91510f165d479db6492d973ade310', 'ffdb3c00f7cc7c25f8240eed3a1d4a50', 'ffe189289e765a331b9b4f04fbfc7bb0', 'ffe4dabd0d2e8075aee4e7db15ce0d80', 'ffe5eaa82debc744b9c9c900fa22bc70', 'ffede07462c60920e4d59e84b65b7030', 'ffe268ffc185abe5b68970e45eb74810', 'fff2e49743064367fc7612168d185480', 'ffe19ece6b385773c362198940f76460', 'fff3236a1ded79b7c1138fca11971f60', 'fff1cc37ea691d75e793c642f6da3340', 'fff9d17e467f93121d15481266a63950', 'ffddcf15d798e8e22b6ec59232ab1fc0', 'ffeb34f7748ac1d42a9ea9e522040480', 'ffddeedc483f0ab09ddc5cb55d93f8f0', 'ffeeb351089c255e152dd2c79c55b250', 'fff26188530a82f3667c66da19168f20', 'ffff0fb5fed019a0

In [ ]:
# the detail info a malware in the dictionary
m_dict['ffff5099125bceaea61251684fbdb770']

{'Parsing Warnings': ['Overlapping offsets in relocation data at RVA: 0xc000'],
 'DOS_HEADER': {'Structure': 'IMAGE_DOS_HEADER',
  'e_magic': {'FileOffset': 0, 'Offset': 0, 'Value': 23117},
  'e_cblp': {'FileOffset': 2, 'Offset': 2, 'Value': 80},
  'e_cp': {'FileOffset': 4, 'Offset': 4, 'Value': 2},
  'e_crlc': {'FileOffset': 6, 'Offset': 6, 'Value': 0},
  'e_cparhdr': {'FileOffset': 8, 'Offset': 8, 'Value': 4},
  'e_minalloc': {'FileOffset': 10, 'Offset': 10, 'Value': 15},
  'e_maxalloc': {'FileOffset': 12, 'Offset': 12, 'Value': 65535},
  'e_ss': {'FileOffset': 14, 'Offset': 14, 'Value': 0},
  'e_sp': {'FileOffset': 16, 'Offset': 16, 'Value': 184},
  'e_csum': {'FileOffset': 18, 'Offset': 18, 'Value': 0},
  'e_ip': {'FileOffset': 20, 'Offset': 20, 'Value': 0},
  'e_cs': {'FileOffset': 22, 'Offset': 22, 'Value': 0},
  'e_lfarlc': {'FileOffset': 24, 'Offset': 24, 'Value': 64},
  'e_ovno': {'FileOffset': 26, 'Offset': 26, 'Value': 26},
  'e_res': {'FileOffset': 28,
   'Offset': 28,
   '

### A dump file example

The above #2 pickle file are generated from pe's dump_dict() funciton.

In [ ]:
# Get a 32-bit putty pe file
!wget https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe -O putty32.exe

--2023-04-15 08:17:55--  https://the.earth.li/~sgtatham/putty/latest/w32/putty.exe
Resolving the.earth.li (the.earth.li)... 93.93.131.124, 2a00:1098:86:4d:c0ff:ee:15:900d
Connecting to the.earth.li (the.earth.li)|93.93.131.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://the.earth.li/~sgtatham/putty/0.78/w32/putty.exe [following]
--2023-04-15 08:17:56--  https://the.earth.li/~sgtatham/putty/0.78/w32/putty.exe
Reusing existing connection to the.earth.li:443.
HTTP request sent, awaiting response... 200 OK
Length: 1477416 (1.4M) [application/x-msdos-program]
Saving to: ‘putty32.exe’

putty32.exe         100%[===================>]   1.41M  2.57MB/s    in 0.5s    

2023-04-15 08:17:57 (2.57 MB/s) - ‘putty32.exe’ saved [1477416/1477416]



In [ ]:
!pip install pefile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip show pefile

Name: pefile
Version: 2023.2.7
Summary: Python PE parsing module
Home-page: https://github.com/erocarrera/pefile
Author: Ero Carrera
Author-email: ero.carrera@gmail.com
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: 
Required-by: 


In [ ]:
import pefile

pef = pefile.PE("/content/putty32.exe", True)
dump_dict = pef.dump_dict()

In [ ]:
# a pefile dump dictionary has some information
# they are stored in different "keys"
print(dump_dict.keys())

dict_keys(['DOS_HEADER', 'NT_HEADERS', 'FILE_HEADER', 'Flags', 'OPTIONAL_HEADER', 'DllCharacteristics', 'PE Sections', 'Directories', 'Version Information', 'Imported symbols', 'Resource directory', 'TLS', 'LOAD_CONFIG', 'Base relocations'])


In [ ]:
# show one of the data in 'DOS_HEADER' key
# and it looks like a python dictionary
# you can put this data in a json or a dictionaary structure for further analysis
print(dump_dict['DOS_HEADER'])

{'Structure': 'IMAGE_DOS_HEADER', 'e_magic': {'FileOffset': 0, 'Offset': 0, 'Value': 23117}, 'e_cblp': {'FileOffset': 2, 'Offset': 2, 'Value': 120}, 'e_cp': {'FileOffset': 4, 'Offset': 4, 'Value': 1}, 'e_crlc': {'FileOffset': 6, 'Offset': 6, 'Value': 0}, 'e_cparhdr': {'FileOffset': 8, 'Offset': 8, 'Value': 4}, 'e_minalloc': {'FileOffset': 10, 'Offset': 10, 'Value': 0}, 'e_maxalloc': {'FileOffset': 12, 'Offset': 12, 'Value': 0}, 'e_ss': {'FileOffset': 14, 'Offset': 14, 'Value': 0}, 'e_sp': {'FileOffset': 16, 'Offset': 16, 'Value': 0}, 'e_csum': {'FileOffset': 18, 'Offset': 18, 'Value': 0}, 'e_ip': {'FileOffset': 20, 'Offset': 20, 'Value': 0}, 'e_cs': {'FileOffset': 22, 'Offset': 22, 'Value': 0}, 'e_lfarlc': {'FileOffset': 24, 'Offset': 24, 'Value': 64}, 'e_ovno': {'FileOffset': 26, 'Offset': 26, 'Value': 0}, 'e_res': {'FileOffset': 28, 'Offset': 28, 'Value': '\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00'}, 'e_oemid': {'FileOffset': 36, 'Offset': 36, 'Value': 0}, 'e_oeminfo': {'FileOffset': 

In [ ]:
# for eample, I put it in a dictionaty
# now I can easily retrive the info stored in it by keys
dos_header_dict = dict(dump_dict['DOS_HEADER'])
print(dos_header_dict.keys())

print(dos_header_dict['Structure'])  # name
print(dos_header_dict['e_magic'])  # e_magic

dict_keys(['Structure', 'e_magic', 'e_cblp', 'e_cp', 'e_crlc', 'e_cparhdr', 'e_minalloc', 'e_maxalloc', 'e_ss', 'e_sp', 'e_csum', 'e_ip', 'e_cs', 'e_lfarlc', 'e_ovno', 'e_res', 'e_oemid', 'e_oeminfo', 'e_res2', 'e_lfanew'])
IMAGE_DOS_HEADER
{'FileOffset': 0, 'Offset': 0, 'Value': 23117}


In [ ]:
# another example

file_header_dict = dict(dump_dict['FILE_HEADER'])
print(file_header_dict.keys())

print(file_header_dict['Machine'])
print(file_header_dict['NumberOfSections'])
print(file_header_dict['TimeDateStamp'])

dict_keys(['Structure', 'Machine', 'NumberOfSections', 'TimeDateStamp', 'PointerToSymbolTable', 'NumberOfSymbols', 'SizeOfOptionalHeader', 'Characteristics'])
{'FileOffset': 124, 'Offset': 0, 'Value': 332}
{'FileOffset': 126, 'Offset': 2, 'Value': 8}
{'FileOffset': 128, 'Offset': 4, 'Value': '0x635C1085 [Fri Oct 28 17:25:25 2022 UTC]'}


In [ ]:
# You can even print all dump_dict to find some valueable infomation.
# pprint stands for pretty print, which makes you easier to view the data
import pprint
pprint.pprint(dump_dict['Imported symbols']) # change this to other keys

[[{'Characteristics': {'FileOffset': 1029208, 'Offset': 0, 'Value': 1036044},
   'FirstThunk': {'FileOffset': 1029224, 'Offset': 16, 'Value': 1037404},
   'ForwarderChain': {'FileOffset': 1029216, 'Offset': 8, 'Value': 0},
   'Name': {'FileOffset': 1029220, 'Offset': 12, 'Value': 1044606},
   'OriginalFirstThunk': {'FileOffset': 1029208, 'Offset': 0, 'Value': 1036044},
   'Structure': 'IMAGE_IMPORT_DESCRIPTOR',
   'TimeDateStamp': {'FileOffset': 1029212,
                     'Offset': 4,
                     'Value': '0x0        [Thu Jan  1 00:00:00 1970 UTC]'}},
  {'DLL': b'GDI32.dll', 'Hint': 19, 'Name': b'BitBlt'},
  {'DLL': b'GDI32.dll', 'Hint': 41, 'Name': b'CreateBitmap'},
  {'DLL': b'GDI32.dll', 'Hint': 48, 'Name': b'CreateCompatibleBitmap'},
  {'DLL': b'GDI32.dll', 'Hint': 49, 'Name': b'CreateCompatibleDC'},
  {'DLL': b'GDI32.dll', 'Hint': 63, 'Name': b'CreateFontA'},
  {'DLL': b'GDI32.dll', 'Hint': 64, 'Name': b'CreateFontIndirectA'},
  {'DLL': b'GDI32.dll', 'Hint': 77, 'Name'

### Analyze the downloaded data

Now you can analyze the 242 malware samples by #1 and/or #2 dump files.

In [ ]:
# The id of the malwares are MD5 in the worm.csv.
# and/or the keys of worm_dict and m_dict.
# Note that you should skip some samples specified in worm.csv.

for key in m_dict.keys():
  print(key)

ffff5099125bceaea61251684fbdb770
fff80ae9338aec1e979ad1ddeb060560
fff000bb386ed5fca21866fab5172500
fff71cb8da44b471b448dad8d3c1ac20
ffe894389225f1d102769d89261acda0
fffda772e9682b5cd713a1aff6bebe71
fff1a3306e69df859b43e71cc2671f00
fffd904f5de5d9ef529b07b3bcf6f5b0
ffde11ff4926d17df917040683095b50
fff27dfe35219a2f7ab157fec9a6da40
ffe91510f165d479db6492d973ade310
ffdb3c00f7cc7c25f8240eed3a1d4a50
ffe189289e765a331b9b4f04fbfc7bb0
ffe4dabd0d2e8075aee4e7db15ce0d80
ffe5eaa82debc744b9c9c900fa22bc70
ffede07462c60920e4d59e84b65b7030
ffe268ffc185abe5b68970e45eb74810
fff2e49743064367fc7612168d185480
ffe19ece6b385773c362198940f76460
fff3236a1ded79b7c1138fca11971f60
fff1cc37ea691d75e793c642f6da3340
fff9d17e467f93121d15481266a63950
ffddcf15d798e8e22b6ec59232ab1fc0
ffeb34f7748ac1d42a9ea9e522040480
ffddeedc483f0ab09ddc5cb55d93f8f0
ffeeb351089c255e152dd2c79c55b250
fff26188530a82f3667c66da19168f20
ffff0fb5fed019a0cdf8c0fc31c40260
ffdf44585f9e8a8e709e5ebe97d4d790
ffdb2677884ac3b6feb1b436747b5f70
fff43e34a6

In [ ]:
# so the fist malware m_dict is
first_dict = m_dict['fff23e2c4f99371e5007aa18a6476120']

# you can now access the data you want
# for example print all sections and their info.
for sec in first_dict['PE Sections']:
  print(sec)

# or like this (as what we see in pefile demonstration)
print("\nName, Misc_VirtualSize, SizeOfRawData, Entropy, MD5")
for sec in first_dict['PE Sections']:
  print(sec['Name']['Value'], sec['Misc_VirtualSize']['Value'], sec['SizeOfRawData']['Value'], sec['Entropy'], sec['MD5'])

{'Structure': 'IMAGE_SECTION_HEADER', 'Name': {'FileOffset': 480, 'Offset': 0, 'Value': '.text\\x00\\x00\\x00'}, 'Misc': {'FileOffset': 488, 'Offset': 8, 'Value': 510}, 'Misc_PhysicalAddress': {'FileOffset': 488, 'Offset': 8, 'Value': 510}, 'Misc_VirtualSize': {'FileOffset': 488, 'Offset': 8, 'Value': 510}, 'VirtualAddress': {'FileOffset': 492, 'Offset': 12, 'Value': 4096}, 'SizeOfRawData': {'FileOffset': 496, 'Offset': 16, 'Value': 512}, 'PointerToRawData': {'FileOffset': 500, 'Offset': 20, 'Value': 1024}, 'PointerToRelocations': {'FileOffset': 504, 'Offset': 24, 'Value': 0}, 'PointerToLinenumbers': {'FileOffset': 508, 'Offset': 28, 'Value': 0}, 'NumberOfRelocations': {'FileOffset': 512, 'Offset': 32, 'Value': 0}, 'NumberOfLinenumbers': {'FileOffset': 514, 'Offset': 34, 'Value': 0}, 'Characteristics': {'FileOffset': 516, 'Offset': 36, 'Value': 1610612768}, 'Flags': ['IMAGE_SCN_CNT_CODE', 'IMAGE_SCN_MEM_EXECUTE', 'IMAGE_SCN_MEM_READ'], 'Entropy': 5.676232928844902, 'MD5': 'e15dfbe2eab6

In [ ]:
# Here you can output the dll and windows api used by a pe file
# You may print dump_dict['Imported symbols'] first
# and try to locate the info you want to retrive.

api_dict = dict()
for records in first_dict['Imported symbols']:
  for record in records:
    if 'Name' in record:
      if isinstance(record['Name'], bytes):
        dll = record['DLL'].decode("ascii")
        name = record['Name'].decode("ascii")
        if dll not in api_dict:
          api_dict[dll] = set()
        api_dict[dll].add(name)

for dll in api_dict:
  print(dll)
  print("", api_dict[dll])

KERNEL32.dll
 {'VirtualAlloc', 'GetProcAddress', 'ReadFile', 'LoadLibraryW', 'CreateFileW'}
MSVCRT.dll
 {'free', '_initterm', 'malloc', '_adjust_fdiv'}


### Extract more information

Now its your turn.
Try to buld a feature table for these 242 malwares.

Hint : these 242 malware samples are from several families. Can you classify or cluster them into groups (w.r.t. family)? You can do supervised learning or unsupervised learning.

I suggest to clean the labels made by Microsoft in the csv file first.

Then, the first try might be creating clusers or perform classificaiton based on the libraries and functions used by the samples, and see if you can create a 'distance function' to known the distance between samples (w.r.t. function calls used). Now, since your have distance, then ML algorithms can be applied.

### Your solution

In [ ]:
my_df = _df
my_df.head()

,MD5,Microsoft,noFamilyName,UPXpacked,skip
0,ffdf141aded498e641b45b0e2188e360,Backdoor:Win32/Berbew.DR,0.0,0,0
1,fff738fcbe3244b4048edb3e289dd450,Backdoor:Win32/Berbew.DR,0.0,0,0
2,ffe268ffc185abe5b68970e45eb74810,Backdoor:Win32/Berbew.I,0.0,0,0
3,fff6a4b8265ea0a0d778697c4096c230,Backdoor:Win32/Delf,0.0,0,1
4,fff1517aebae3c980f13efd7edea2480,Backdoor:Win32/Delf.DU,0.0,0,0


##### 先去除空值

In [ ]:
my_df = my_df.dropna(axis=0).reset_index(drop=True)
my_df.head()

,MD5,Microsoft,noFamilyName,UPXpacked,skip
0,ffdf141aded498e641b45b0e2188e360,Backdoor:Win32/Berbew.DR,0.0,0,0
1,fff738fcbe3244b4048edb3e289dd450,Backdoor:Win32/Berbew.DR,0.0,0,0
2,ffe268ffc185abe5b68970e45eb74810,Backdoor:Win32/Berbew.I,0.0,0,0
3,fff6a4b8265ea0a0d778697c4096c230,Backdoor:Win32/Delf,0.0,0,1
4,fff1517aebae3c980f13efd7edea2480,Backdoor:Win32/Delf.DU,0.0,0,0


skip可跳過，=1刪除

UPXpacked老師解壓了可不在意，刪除整欄

noFamilyName =1 刪除


In [ ]:
my_df = my_df.drop(my_df[my_df['skip']==1].index, axis=0).reset_index(drop=True)
my_df = my_df.drop(my_df[my_df['noFamilyName']==1].index, axis=0).reset_index(drop=True)
my_df.head()

,MD5,Microsoft,noFamilyName,UPXpacked,skip
0,ffdf141aded498e641b45b0e2188e360,Backdoor:Win32/Berbew.DR,0.0,0,0
1,fff738fcbe3244b4048edb3e289dd450,Backdoor:Win32/Berbew.DR,0.0,0,0
2,ffe268ffc185abe5b68970e45eb74810,Backdoor:Win32/Berbew.I,0.0,0,0
3,fff1517aebae3c980f13efd7edea2480,Backdoor:Win32/Delf.DU,0.0,0,0
4,fff25b0344d0a9029170375b867ec950,Backdoor:Win32/Delf.DU,0.0,0,0


In [ ]:
my_df = my_df.drop(['noFamilyName', 'UPXpacked', 'skip'], axis=1).reset_index(drop=True)
my_df

,MD5,Microsoft
0,ffdf141aded498e641b45b0e2188e360,Backdoor:Win32/Berbew.DR
1,fff738fcbe3244b4048edb3e289dd450,Backdoor:Win32/Berbew.DR
2,ffe268ffc185abe5b68970e45eb74810,Backdoor:Win32/Berbew.I
3,fff1517aebae3c980f13efd7edea2480,Backdoor:Win32/Delf.DU
4,fff25b0344d0a9029170375b867ec950,Backdoor:Win32/Delf.DU
...,...,...
101,ffeaa3f8551c1bbd77d39668e7690e70,Worm:Win32/Vobfus.gen!I
102,ffddeedc483f0ab09ddc5cb55d93f8f0,Worm:Win32/Vobfus.gen!N
103,ffe5a16e686d28ccc0d53059d24bf370,Worm:Win32/Vobfus.gen!N
104,ffff0fb5fed019a0cdf8c0fc31c40260,Worm:Win32/Wenper


#### clean the labels made by Microsoft in the csv file first.

In [ ]:
Microsoft_keys = list(set(my_df['Microsoft']))

In [ ]:
Microsoft_dict = {}
for i, v in enumerate(Microsoft_keys):
    Microsoft_dict[v] = i
Microsoft_dict

{'Worm:Win32/Gamarue.AB': 0,
 'PWS:Win32/Frethog.AG.dll': 1,
 'Worm:Win32/Vobfus.DA': 2,
 'TrojanDownloader:Win32/Upatre.O': 3,
 'Worm:Win32/Gamarue!rfn': 4,
 'Worm:Win32/Gamarue.X': 5,
 'Worm:Win32/Wenper': 6,
 'Backdoor:Win32/Berbew.I': 7,
 'Virus:Win32/Virut.K': 8,
 'Worm:Win32/Fesber.A': 9,
 'Worm:Win32/Mydoom.O@mm': 10,
 'Backdoor:Win32/PcClient.ZR': 11,
 'Worm:Win32/Pondfull.B': 12,
 'Worm:Win32/Eggnog.A': 13,
 'Worm:Win32/Soltern!rfn': 14,
 'Worm:Win32/Ganelp.E': 15,
 'Worm:Win32/Vobfus.gen!N': 16,
 'Worm:Win32/Fasong': 17,
 'Worm:Win32/Gamarue.Z': 18,
 'Worm:Win32/Picsys.C': 19,
 'Virus:Win32/Viking.KJ': 20,
 'Trojan:Win32/Toga!rfn': 21,
 'TrojanDownloader:Win32/Upatre.L': 22,
 'Backdoor:Win32/Delf.DU': 23,
 'TrojanDropper:Win32/Loring': 24,
 'Virus:Win32/Mabezat.B': 25,
 'Worm:Win32/Crastic.gen!A': 26,
 'TrojanDownloader:Win32/Upatre.A': 27,
 'Trojan:Win32/Beaugrit.gen!AAA': 28,
 'Virus:Win32/Shodi.I': 29,
 'Worm:Win32/Vobfus.gen!I': 30,
 'Worm:Win32/Soltern.L': 31,
 'Worm:Win

In [ ]:
my_df['Microsoft'] = my_df['Microsoft'].map(Microsoft_dict)
my_df

,MD5,Microsoft
0,ffdf141aded498e641b45b0e2188e360,51
1,fff738fcbe3244b4048edb3e289dd450,51
2,ffe268ffc185abe5b68970e45eb74810,7
3,fff1517aebae3c980f13efd7edea2480,23
4,fff25b0344d0a9029170375b867ec950,23
...,...,...
101,ffeaa3f8551c1bbd77d39668e7690e70,30
102,ffddeedc483f0ab09ddc5cb55d93f8f0,16
103,ffe5a16e686d28ccc0d53059d24bf370,16
104,ffff0fb5fed019a0cdf8c0fc31c40260,6


#### the first try might be creating clusers or perform classificaiton based on the libraries and functions used by the samples

In [ ]:
list(worm_dict['fffcb69168b0a0050fb540814832a1b0'].keys())

['KERNEL32.DLL',
 'ADVAPI32.dll',
 'COMCTL32.dll',
 'comdlg32.dll',
 'GDI32.dll',
 'MPR.dll',
 'ole32.dll',
 'OLEAUT32.dll',
 'SHELL32.dll',
 'USER32.dll',
 'VERSION.dll',
 'WINMM.dll',
 'WSOCK32.dll']

In [ ]:
dlls = []

for md5 in my_df['MD5']:
    if worm_dict.get(md5) != None:
        for key in list(worm_dict[md5].keys()):
            if key not in dlls:
                dlls.append(key)

dlls

['ole32.DLL',
 'OLEAUT32.DLL',
 'WININET.DLL',
 'KERNEL32.DLL',
 'USER32.DLL',
 'GDI32.DLL',
 'ADVAPI32.DLL',
 'CRTDLL.DLL',
 'kernel32.dll',
 'user32.dll',
 'advapi32.dll',
 'oleaut32.dll',
 'mpr.dll',
 'wsock32.dll',
 'shell32.dll',
 'wininet.dll',
 'URLMON.DLL',
 'ADVAPI32.dll',
 'MSVCRT.dll',
 'USER32.dll',
 'KERNEL32.dll',
 'WININET.dll',
 'urlmon.dll',
 'WS2_32.dll',
 'iphlpapi.dll',
 'msvcrt.dll',
 'SHELL32.DLL',
 'comctl32.dll',
 'gdi32.dll',
 'version.dll',
 'SETUPAPI.dll',
 'WINSPOOL.DRV',
 'GDI32.dll',
 'OLE32.dll',
 'dhcpcsvc.dll',
 'COMDLG32.dll',
 'SHFOLDER.dll',
 'COMCTL32.dll',
 'MPR.dll',
 'ole32.dll',
 'OLEAUT32.dll',
 'PSAPI.DLL',
 'SHELL32.dll',
 'USERENV.dll',
 'VERSION.dll',
 'WINMM.dll',
 'WSOCK32.dll',
 'NTDLL.dll',
 'MSFEEDIA.dll',
 'MSVCP80.dll',
 'MSVCR80.dll',
 'MSVCIRT.dll',
 'winmm.dll',
 'comdlg32.dll',
 'SHLWAPI.dll',
 'shlwapi.dll']

In [ ]:
my_df[dlls]=0
my_df

,MD5,Microsoft,ole32.DLL,OLEAUT32.DLL,WININET.DLL,KERNEL32.DLL,USER32.DLL,GDI32.DLL,ADVAPI32.DLL,CRTDLL.DLL,...,WSOCK32.dll,NTDLL.dll,MSFEEDIA.dll,MSVCP80.dll,MSVCR80.dll,MSVCIRT.dll,winmm.dll,comdlg32.dll,SHLWAPI.dll,shlwapi.dll
0,ffdf141aded498e641b45b0e2188e360,51,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,fff738fcbe3244b4048edb3e289dd450,51,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ffe268ffc185abe5b68970e45eb74810,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,fff1517aebae3c980f13efd7edea2480,23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,fff25b0344d0a9029170375b867ec950,23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,ffeaa3f8551c1bbd77d39668e7690e70,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102,ffddeedc483f0ab09ddc5cb55d93f8f0,16,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103,ffe5a16e686d28ccc0d53059d24bf370,16,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104,ffff0fb5fed019a0cdf8c0fc31c40260,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for md5 in my_df['MD5']:
    if md5 in worm_dict and worm_dict[md5]:
        for key in worm_dict[md5].keys():
            if key in my_df.columns:
                my_df.loc[my_df['MD5'] == md5, key] = 1
my_df = my_df.set_index(my_df.columns[0])
my_df.head()

,Microsoft,ole32.DLL,OLEAUT32.DLL,WININET.DLL,KERNEL32.DLL,USER32.DLL,GDI32.DLL,ADVAPI32.DLL,CRTDLL.DLL,kernel32.dll,...,WSOCK32.dll,NTDLL.dll,MSFEEDIA.dll,MSVCP80.dll,MSVCR80.dll,MSVCIRT.dll,winmm.dll,comdlg32.dll,SHLWAPI.dll,shlwapi.dll
MD5,,,,,,,,,,,,,,,,,,,,,
ffdf141aded498e641b45b0e2188e360,51,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
fff738fcbe3244b4048edb3e289dd450,51,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
ffe268ffc185abe5b68970e45eb74810,7,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
fff1517aebae3c980f13efd7edea2480,23,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
fff25b0344d0a9029170375b867ec950,23,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


#### 還沒加入m_dict特徵的kmeans分群結果&用knn做的預測ACC

In [ ]:
predict1 = my_df.copy() # 淺拷貝
predict1['kmeans'] = 0

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

kmeans = KMeans(n_clusters=2, random_state=0).fit(my_df)
cluster = kmeans.predict(my_df)
for i in range(2):
  print("第", i+1, "個 cluster 有:")
  for j in my_df[cluster == i].index:
    predict1.loc[j]['kmeans'] = i # 將y值設為分群值
    print(j)
  print('-------------------------')


第 1 個 cluster 有:
ffe268ffc185abe5b68970e45eb74810
fff1517aebae3c980f13efd7edea2480
fff25b0344d0a9029170375b867ec950
ffe48b5a702ce1e7ea3c3851d56afe60
fffd8ca32e739f1d8ad32404b56f6ac0
ffed8955262630e6c953bd459f600600
ffe6abdc72d8a79737a0672519e45e00
fff5042ad92e80aaea76e9d870035ae0
fffbd4a6e48a16038864b418d2f290f0
fff429f8230f3507f720a6c0b97ccc60
ffe9bdfbe933f14cf930330cf983ff30
ffdabec2455576fdce67e5bb7ee0e110
ffe74ad543c00f84ab1ccff4a978ed10
fff3357d3bf495777ed8909a2a6fdcf0
fff64fb9ecf74f016b8e33639a70eef0
fff8df176184edc60804f17e75d3e440
fff9bbf76d90662acdcc3d812a723460
ffe2a4b63d0ec54823d7d783118ed2a0
ffdb3c00f7cc7c25f8240eed3a1d4a50
fffc302c9c02a3e76c5ccdbd97e779f0
fff26188530a82f3667c66da19168f20
ffddcf15d798e8e22b6ec59232ab1fc0
ffde11ff4926d17df917040683095b50
ffde3dd8959bd513756d926fa1e61a50
ffdf44585f9e8a8e709e5ebe97d4d790
ffe4dabd0d2e8075aee4e7db15ce0d80
ffe9afc0f1344eb2ee6b1c391445bdd0
fff0d7e29d50d545cbb5533726b1a360
fff8dd379bdb546af37b9d4f221ac300
fff9d17e467f93121d15481266

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = predict1.iloc[:, :-1]
y = predict1.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

k = 2
knn = KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)

y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('只做file1的Accuracy: {:.2f}'.format(accuracy))


只做file1的Accuracy: 0.97


#### 加入m_dict特徵的kmeans分群結果&用knn做的預測ACC

In [ ]:
predict2 = my_df.copy() # 淺拷貝

In [ ]:
import re

first_dict = m_dict[predict2.index[0]]
print("\nName, Misc_VirtualSize, SizeOfRawData, Entropy")
for sec in first_dict['PE Sections']:
  print(re.search(r"^.*?\\", sec['Name']['Value']).group(0)[:-1],'/', sec['Misc_VirtualSize']['Value'], '/',sec['SizeOfRawData']['Value'], '/',sec['Entropy'])


Name, Misc_VirtualSize, SizeOfRawData, Entropy
.text / 32428 / 32428 / 7.111425801329107
.bss / 136112 / 0 / 0.0
.data / 12752 / 12752 / 6.071940380883166
.idata / 3748 / 3748 / 5.0788658748287245
.aciof / 4096 / 512 / 2.270456150351499


In [ ]:
name_list = []
for i in range(len(predict2)):
  first_dict = m_dict[predict2.index[i]]
  for sec in first_dict['PE Sections']:
    if re.search(r"^.*?\\", sec['Name']['Value']).group(0)[:-1] not in name_list:
      name_list.append(re.search(r"^.*?\\", sec['Name']['Value']).group(0)[:-1])
predict2[name_list] = 0

In [ ]:
for i in range(len(predict2)):
    first_dict = m_dict[predict2.index[i]]
    for sec in first_dict['PE Sections']:
        s = re.search(r"^.*?\\", sec['Name']['Value']).group(0)[:-1]
        # predict2.iloc[i][s] = 1

        predict2.loc[predict2.index[i], s] = 1
predict2.head()

,Microsoft,ole32.DLL,OLEAUT32.DLL,WININET.DLL,KERNEL32.DLL,USER32.DLL,GDI32.DLL,ADVAPI32.DLL,CRTDLL.DLL,kernel32.dll,...,.CRT,sdata,.aspack,.adata,ExeS,UPX0,UPX1,dtbjzoi,.debug,.edata
MD5,,,,,,,,,,,,,,,,,,,,,
ffdf141aded498e641b45b0e2188e360,51,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
fff738fcbe3244b4048edb3e289dd450,51,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
ffe268ffc185abe5b68970e45eb74810,7,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
fff1517aebae3c980f13efd7edea2480,23,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
fff25b0344d0a9029170375b867ec950,23,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
predict2['kmeans'] = 0
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

kmeans = KMeans(n_clusters=2, random_state=0).fit(predict2)
cluster = kmeans.predict(predict2)
for i in range(2):
  print("第", i+1, "個 cluster 有:")
  for j in predict2[cluster == i].index:
    predict2.loc[j]['kmeans'] = i # 將y值設為分群值
    print(j)
  print('-------------------------')


第 1 個 cluster 有:
ffe268ffc185abe5b68970e45eb74810
fff1517aebae3c980f13efd7edea2480
fff25b0344d0a9029170375b867ec950
ffe48b5a702ce1e7ea3c3851d56afe60
fffd8ca32e739f1d8ad32404b56f6ac0
ffed8955262630e6c953bd459f600600
ffe6abdc72d8a79737a0672519e45e00
fff5042ad92e80aaea76e9d870035ae0
fffbd4a6e48a16038864b418d2f290f0
fff429f8230f3507f720a6c0b97ccc60
ffe9bdfbe933f14cf930330cf983ff30
ffdabec2455576fdce67e5bb7ee0e110
ffe74ad543c00f84ab1ccff4a978ed10
fff3357d3bf495777ed8909a2a6fdcf0
fff64fb9ecf74f016b8e33639a70eef0
fff8df176184edc60804f17e75d3e440
fff9bbf76d90662acdcc3d812a723460
ffe2a4b63d0ec54823d7d783118ed2a0
ffdb3c00f7cc7c25f8240eed3a1d4a50
fffc302c9c02a3e76c5ccdbd97e779f0
fff26188530a82f3667c66da19168f20
ffddcf15d798e8e22b6ec59232ab1fc0
ffde11ff4926d17df917040683095b50
ffde3dd8959bd513756d926fa1e61a50
ffdf44585f9e8a8e709e5ebe97d4d790
ffe4dabd0d2e8075aee4e7db15ce0d80
ffe9afc0f1344eb2ee6b1c391445bdd0
fff0d7e29d50d545cbb5533726b1a360
fff8dd379bdb546af37b9d4f221ac300
fff9d17e467f93121d15481266

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
predict2.head()

,Microsoft,ole32.DLL,OLEAUT32.DLL,WININET.DLL,KERNEL32.DLL,USER32.DLL,GDI32.DLL,ADVAPI32.DLL,CRTDLL.DLL,kernel32.dll,...,sdata,.aspack,.adata,ExeS,UPX0,UPX1,dtbjzoi,.debug,.edata,kmeans
MD5,,,,,,,,,,,,,,,,,,,,,
ffdf141aded498e641b45b0e2188e360,51,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
fff738fcbe3244b4048edb3e289dd450,51,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
ffe268ffc185abe5b68970e45eb74810,7,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
fff1517aebae3c980f13efd7edea2480,23,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
fff25b0344d0a9029170375b867ec950,23,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = predict2.iloc[:, :-1]
y = predict2.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

k = 2
knn = KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)

y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('做file1+file2的Accuracy: {:.2f}'.format(accuracy))


做file1+file2的Accuracy: 0.97


In [ ]:
diff = predict1['kmeans'].compare(predict2['kmeans'])
print("結果不同的數量", len(diff.columns))

結果不同的數量 2
